In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


นาย ภาสุ พงศาวกุล 6410210245  ผมเปลี่ยน dataset นะครับอาจารย์

https://drive.google.com/drive/folders/1eiIkEkKf6jZRzH3i3DZ-I0RB8n1j4xoT?usp=sharing

# 1. การเตรียมข้อมูล (Preparing Data)


##  feature extraction

In [ ]:
import os
import librosa # export audio
import numpy as np # padding
import matplotlib.pyplot as plt # visualisation
from IPython.display import Audio # display tester audio
from scipy.ndimage import gaussian_filter1d # smoothing the chromagram to remove noises
from sklearn.model_selection import train_test_split # split the dataset train & test
from keras.models import Sequential # cnn model frame
from tensorflow.keras.callbacks import EarlyStopping # stop
from keras.utils import to_categorical # to create hot labels
import IPython.display

In [ ]:
curr_freq = 55
freq_list = []

for i in range(96):
    freq_list.append(curr_freq)
    curr_freq *= np.power(2, 1/12)

#reshaping and creating dataframe
freq_array = np.reshape(np.round(freq_list,1), (8, 12))
cols = ["A", "A#", "B", "C", "C#", "D", "D#", "E", "F", "F#", "G", "G#"]
df_note_freqs = pd.DataFrame(freq_array, columns=cols)
print("NOTE FREQUENCIES IN WESTERN MUSIC")
df_note_freqs.head(10)

NOTE FREQUENCIES IN WESTERN MUSIC


,A,A#,B,C,C#,D,D#,E,F,F#,G,G#
0,55.0,58.3,61.7,65.4,69.3,73.4,77.8,82.4,87.3,92.5,98.0,103.8
1,110.0,116.5,123.5,130.8,138.6,146.8,155.6,164.8,174.6,185.0,196.0,207.7
2,220.0,233.1,246.9,261.6,277.2,293.7,311.1,329.6,349.2,370.0,392.0,415.3
3,440.0,466.2,493.9,523.3,554.4,587.3,622.3,659.3,698.5,740.0,784.0,830.6
4,880.0,932.3,987.8,1046.5,1108.7,1174.7,1244.5,1318.5,1396.9,1480.0,1568.0,1661.2
5,1760.0,1864.7,1975.5,2093.0,2217.5,2349.3,2489.0,2637.0,2793.8,2960.0,3136.0,3322.4
6,3520.0,3729.3,3951.1,4186.0,4434.9,4698.6,4978.0,5274.0,5587.7,5919.9,6271.9,6644.9
7,7040.0,7458.6,7902.1,8372.0,8869.8,9397.3,9956.1,10548.1,11175.3,11839.8,12543.9,13289.8


In [ ]:
import numpy as np
import pandas as pd

curr_freq = 55
freq_list = []

for i in range(96):
    freq_list.append(curr_freq)
    curr_freq *= np.power(2, 1/12)

# Reshaping and creating dataframe
freq_array = np.reshape(np.round(freq_list, 1), (8, 12))
cols = ["A", "A#", "B", "C", "C#", "D", "D#", "E", "F", "F#", "G", "G#"]
df_note_freqs = pd.DataFrame(freq_array, columns=cols)

# Save as CSV file
df_note_freqs.to_csv('note_frequencies.csv', index=False)


In [ ]:
path_1 = "/content/drive/MyDrive/Machine Learning/dataset/Audio_Files/Major/Major_0.wav"

IPython.display.Audio(path_1, rate = 44100)

In [ ]:
path_2 = "/content/drive/MyDrive/Machine Learning/dataset/Audio_Files/Minor/Minor_0.wav"

IPython.display.Audio(path_2, rate = 44100)

In [ ]:
import librosa
import numpy as np
from scipy.stats import kurtosis, skew

def extract_audio_features(file_path):
    # Load audio file
    y, sr = librosa.load(file_path, sr=None)

    # Extract features
    # Spectral Centroid
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)[0]

    # Spectral Bandwidth
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)[0]

    # Zero-Crossing Rate
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y)[0]

    # Mel-Frequency Cepstral Coefficients (MFCCs)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    mfccs_mean = np.mean(mfccs, axis=1)

    # Statistical features
    mean = np.mean(y)
    median = np.median(y)
    std = np.std(y)
    skewness = skew(y)
    kurt = kurtosis(y)

    # Concatenate all features
    features = np.concatenate([spectral_centroid, spectral_bandwidth, zero_crossing_rate, mfccs_mean, [mean, median, std, skewness, kurt]])

    return features

# Example usage
file_path = "/content/drive/MyDrive/Machine Learning/dataset/Audio_Files/Major/Major_0.wav"
audio_features = extract_audio_features(file_path)
print("Extracted audio features shape:", audio_features.shape)


Extracted audio features shape: (600,)


In [ ]:
# labelling and gathering audio path
path = "/content/drive/MyDrive/Machine Learning/dataset/Audio_Files"
audio_path = []
labels = []
# counter to count minor and major wav file
counter = 0
# labs -> labelling (major = 0, minor = 1)
labs = 0
for x in os.listdir(path):
    subpath = os.path.join(path, x)
    for audios in os.listdir(subpath):
        audio_path.append(os.path.join(subpath, audios))
        labels.append(labs)
    labs+=1
    if counter == 0:
        counter = len(audio_path)
        print(f"number of {x} audio: {counter}")
    else:
        print(f"number of {x} audio: {len(audio_path)-counter}")

number of Minor audio: 357
number of Major audio: 502


In [ ]:
# in here we have audio path and labels, and need to extract the chroma feature
# split
X_train, X_val, y_train, y_val = train_test_split(audio_path, labels, test_size = 0.3, random_state = 32)
print(f"first val before splitted contains:\nX: {len(X_val)}\ny: {len(y_val)}")

split = len(y_val)//3
X_test = X_val[:split]
y_test = y_val[:split]

print(f"test set contains:\nX: {len(X_test)}\ny: {len(y_test)}")

X_val = X_val[split:]
y_val = y_val[split:]
print(f"val set contains:\nX: {len(X_val)}\ny: {len(y_val)}")

first val before splitted contains:
X: 258
y: 258
test set contains:
X: 86
y: 86
val set contains:
X: 172
y: 172


# (Model Selection)


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.datasets import load_iris


##model SVC

In [ ]:
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

# Function to extract features from audio files
def extract_features(audio_path):
    features = []
    for path in audio_path:
        try:
            # Load audio file
            y, sr = librosa.load(path, sr=None)
            # Extract feature (e.g., MFCCs)
            mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
            # Flatten the MFCCs matrix and take mean
            features.append(np.mean(mfccs, axis=1))
        except Exception as e:
            print(f"Error processing {path}: {str(e)}")
    return np.array(features)


# Extract features from audio files
X = extract_features(audio_path)

# Split the data into training, validation, and test sets
X_train, X_val, y_train, y_val = train_test_split(X, labels, test_size=0.3, random_state=32)
split = len(y_val) // 3
X_test = X_val[:split]
y_test = y_val[:split]
X_val = X_val[split:]
y_val = y_val[split:]

# Create and train SVM model
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train, y_train)

# Evaluate the model on the test set
accuracy = svm_model.score(X_test, y_test)
print("Accuracy on test set:", accuracy)

Accuracy on test set: 0.627906976744186


##model ANN

In [ ]:
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical

# Function to extract features from audio files
def extract_features(audio_path):
    features = []
    for path in audio_path:
        # Load audio file
        y, sr = librosa.load(path, sr=None)
        # Extract feature (e.g., MFCCs)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        # Flatten the MFCCs matrix and take mean
        features.append(np.mean(mfccs, axis=1))
    return np.array(features)

# Extract features from audio files
X = extract_features(audio_path)

# Split the data into training, validation, and test sets
X_train, X_val, y_train, y_val = train_test_split(X, labels, test_size=0.3, random_state=32)
split = len(y_val) // 3
X_test = X_val[:split]
y_test = y_val[:split]
X_val = X_val[split:]
y_val = y_val[split:]

# Convert labels to one-hot encoded format
num_classes = len(np.unique(labels))
y_train = to_categorical(y_train, num_classes)
y_val = to_categorical(y_val, num_classes)
y_test = to_categorical(y_test, num_classes)

# Create the ANN model
model = Sequential([
    Flatten(input_shape=(X_train.shape[1],)),  # Flatten the input
    Dense(128, activation='relu'),             # Dense layer with 128 units and ReLU activation
    Dense(64, activation='relu'),              # Dense layer with 64 units and ReLU activation
    Dense(num_classes, activation='softmax')   # Output layer with softmax activation
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print("Accuracy on test set:", accuracy)

# Predict on the test set
y_pred = model.predict(X_test)


Epoch 1/10
19/19 [==============================] - 1s 20ms/step - loss: 4.0252 - accuracy: 0.5108 - val_loss: 3.0107 - val_accuracy: 0.5640
Epoch 2/10
19/19 [==============================] - 0s 7ms/step - loss: 1.6198 - accuracy: 0.5374 - val_loss: 0.9933 - val_accuracy: 0.5523
Epoch 3/10
19/19 [==============================] - 0s 7ms/step - loss: 1.2128 - accuracy: 0.5141 - val_loss: 1.4679 - val_accuracy: 0.5640
Epoch 4/10
19/19 [==============================] - 0s 6ms/step - loss: 1.2143 - accuracy: 0.5541 - val_loss: 1.8266 - val_accuracy: 0.5640
Epoch 5/10
19/19 [==============================] - 0s 8ms/step - loss: 1.6840 - accuracy: 0.5141 - val_loss: 1.4859 - val_accuracy: 0.5930
Epoch 6/10
19/19 [==============================] - 0s 10ms/step - loss: 1.2203 - accuracy: 0.5491 - val_loss: 0.9169 - val_accuracy: 0.5581
Epoch 7/10
19/19 [==============================] - 0s 5ms/step - loss: 1.1181 - accuracy: 0.5275 - val_loss: 0.8784 - val_accuracy: 0.4884
Epoch 8/10
19/19 [

#อธิบายและวิเคราะห์ผลการประเมินประสิทธิภาพของทั้ง 2 โมเดลที่ได้


โมเดลแรกที่คุณฝึกอบรมมีค่าความแม่นยำบนชุดทดสอบประมาณ 0.63 หรือ 63% ซึ่งหมายความว่าโมเดลสามารถทำนายคลาสของข้อมูลในชุดทดสอบได้ถูกต้องประมาณ 63% ของเวลา

สำหรับโมเดลที่สอง มีค่าความแม่นยำบนชุดทดสอบประมาณ 0.64 หรือ 64% ซึ่งสูงกว่าโมเดลแรกเล็กน้อย เมื่อเปรียบเทียบกับโมเดลแรก โมเดลนี้อาจจะไม่ได้รับความเสี่ยงต่ำจากการ overfitting เท่ากับโมเดลแรก แต่ยังคงมีความแม่นยำในการทำนายที่สูงกว่าโมเดลแรก

#  (Deploy Model)

##บันทึก Model ที่ดีกว่า


In [ ]:
import joblib

# Save the SVM model
joblib.dump(svm_model, "/content/drive/MyDrive/Machine Learning/dataset/Audio_Files/svm_model.pkl")



['/content/drive/MyDrive/Machine Learning/dataset/Audio_Files/svm_model.pkl']

# ทดสอบ model

In [ ]:
import librosa
import numpy as np
from sklearn.svm import SVC
import joblib
from keras.models import load_model

# Function to extract features from audio file
def extract_features(audio_path):
    try:
        # Load audio file
        y, sr = librosa.load(audio_path, sr=None)
        # Extract feature (e.g., MFCCs)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        # Flatten the MFCCs matrix and take mean
        features = np.mean(mfccs, axis=1)
        return features.reshape(1, -1)  # Reshape to fit the model input
    except Exception as e:
        print(f"Error processing {audio_path}: {str(e)}")
        return None

# Load the SVM model
svm_model = joblib.load("/content/drive/MyDrive/Machine Learning/dataset/Audio_Files/svm_model.pkl")

# Path to the audio file you want to classify
audio_file_path = "/content/drive/MyDrive/Machine Learning/dataset/Audio_Files/Major/Major_0.wav"

# Extract features from the audio file
audio_features = extract_features(audio_file_path)

# Make prediction
if audio_features is not None:
    prediction = svm_model.predict(audio_features)
    print("Prediction:", prediction)
else:
    print("Failed to extract features from the audio file.")


1/1 [==============================] - 0s 240ms/step
Prediction: [[0.00640971 0.99359035]]
